# ML Pipeline 
按照如下的指导要求，搭建你的机器学习管道。
### 1. 导入与加载
- 导入 Python 库
- 使用 [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html) 从数据库中加载数据集
- 定义特征变量X 和目标变量 Y

In [49]:
# import libraries
# TODO 1
import pandas as pd
import numpy as np
import re
import pickle
#NLTK libraries
import nltk
nltk.download(['punkt', 'wordnet', 'stopwords', 'averaged_perceptron_tagger'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sqlalchemy import create_engine

from sklearn.metrics import classification_report,precision_recall_fscore_support
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\wyf35\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\wyf35\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\wyf35\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\wyf35\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [52]:
# load data from database
# TODO 2
engine = create_engine('sqlite:///../data/DisasterResponse.db')
df = pd.read_sql_table('DisasterResponsePipeline_table',engine)
X = df['message']
Y = df.iloc[:, 4:]
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
X.head()

,message,original,genre
0,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [13]:
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. 编写分词函数，开始处理文本

In [14]:
# test_text = X.head(1)
X[0]

'Weather update - a cold front from Cuba that could pass over Haiti'

In [53]:
def tokenize(text):
    # 使用正则表达式清理数据
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    tokens = word_tokenize(text)
    
    # lemmatizer并去掉停用词
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        if tok not in nltk.corpus.stopwords.words('english'):
            clean_tok = lemmatizer.lemmatize(tok).strip()
            clean_tokens.append(clean_tok)

    return clean_tokens


In [54]:
text = X[0]
# tokenize(text)
print(text)
tokenize(text)

Weather update - a cold front from Cuba that could pass over Haiti


['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']

### 3. 创建机器学习管道 
这个机器学习管道应该接收 `message` 列作输入，输出分类结果，分类结果属于该数据集中的 36 个类。你会发现 [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) 在预测多目标变量时很有用。

In [55]:
#创建一个机器学习管道
pipeline = Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer()),
                ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=42)))
            ])

### 4. 训练管道
- 将数据分割成训练和测试集
- 训练管道

In [58]:
# 分割数据为训练和测试集
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = .20, random_state=42)

#训练数据来拟合模型
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                   

### 5. 测试模型
报告数据集中每个输出类别的 f1 得分、准确度和召回率。你可以对列进行遍历，并对每个元素调用 sklearn 的 `classification_report`。

In [59]:
# 使用测试数据进行预测
#得到当前模型下的对测试数据集的预测结果
Y_pred_test = pipeline.predict(X_test)


In [60]:
#classification_report函数用于显示主要分类指标的文本报告．
#在报告中显示每个类的精确度，召回率，F1值等信息
Y_pred_test = pd.DataFrame(Y_pred_test, columns = Y_test.columns)
for column in Y_test.columns:
    print('类别： {}'.format(column))
#     target_names = Y.columns
    print(classification_report(Y_test[column], Y_pred_test[column]))

类别： related
              precision    recall  f1-score   support

           0       0.71      0.42      0.53      1266
           1       0.83      0.94      0.88      3938
           2       0.39      0.45      0.42        40

    accuracy                           0.81      5244
   macro avg       0.64      0.60      0.61      5244
weighted avg       0.80      0.81      0.79      5244

类别： request
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      4349
           1       0.83      0.48      0.61       895

    accuracy                           0.89      5244
   macro avg       0.87      0.73      0.77      5244
weighted avg       0.89      0.89      0.88      5244

类别： offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5218
           1       0.00      0.00      0.00        26

    accuracy                           1.00      5244
   macro avg       0.50      0.50      0

c:\users\wyf35\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



类别： child_alone
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5244

    accuracy                           1.00      5244
   macro avg       1.00      1.00      1.00      5244
weighted avg       1.00      1.00      1.00      5244

类别： water
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      4905
           1       0.85      0.37      0.52       339

    accuracy                           0.96      5244
   macro avg       0.91      0.69      0.75      5244
weighted avg       0.95      0.96      0.95      5244

类别： food
              precision    recall  f1-score   support

           0       0.95      0.99      0.97      4649
           1       0.85      0.60      0.71       595

    accuracy                           0.94      5244
   macro avg       0.90      0.79      0.84      5244
weighted avg       0.94      0.94      0.94      5244

类别： shelter
              precision  

              precision    recall  f1-score   support

           0       0.86      0.98      0.92      4223
           1       0.78      0.34      0.48      1021

    accuracy                           0.85      5244
   macro avg       0.82      0.66      0.70      5244
weighted avg       0.85      0.85      0.83      5244



### 6. 优化模型
使用网格搜索来找到最优的参数组合。 

In [64]:
# 设置参数
parameters = {
    'tfidf__norm': ['l1', 'l2'],
    'tfidf__sublinear_tf': [True, False]
}

# Instantiating a GridSearch method to identify an optimised model
cv = GridSearchCV(pipeline, param_grid=parameters, cv=5, verbose=50) 

### 7. 测试模型
打印微调后的模型的精确度、准确率和召回率。  

因为本项目主要关注代码质量、开发流程和管道技术，所有没有模型性能指标的最低要求。但是，微调模型提高精确度、准确率和召回率可以让你的项目脱颖而出——特别是让你的简历更出彩。

In [65]:

cv.fit(X_train, Y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV] tfidf__norm=l1, tfidf__sublinear_tf=True ........................
[CV]  tfidf__norm=l1, tfidf__sublinear_tf=True, score=0.253, total=14.1min
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 14.1min remaining:    0.0s
[CV] tfidf__norm=l1, tfidf__sublinear_tf=True ........................
[CV]  tfidf__norm=l1, tfidf__sublinear_tf=True, score=0.261, total= 9.9min
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 24.0min remaining:    0.0s
[CV] tfidf__norm=l1, tfidf__sublinear_tf=True ........................
[CV]  tfidf__norm=l1, tfidf__sublinear_tf=True, score=0.269, total= 9.6min
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed: 33.6min remaining:    0.0s
[CV] tfidf__norm=l1, tfidf__sublinear_tf=True ........................


KeyboardInterrupt: 

In [ ]:
Y_pred_grid_pd1 = pd.DataFrame(Y_pred_test1, columns = Y_test.columns)
for column in Y_test.columns:
    print('------------------------------------------------------\n')
    print('FEATURE: {}\n'.format(column))
    print(classification_report(Y_test[column],Y_pred_grid_pd1[column]))

### 8. 继续优化模型，比如：
* 尝试其他的机器学习算法
* 尝试除 TF-IDF 外其他的特征

In [ ]:
pipeline1 = Pipeline([('cvect', CountVectorizer(tokenizer = tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('clf_2', MultiOutputClassifier(AdaBoostClassifier()))
                     ])

pipelin1.fit(X_train, y_train)

In [ ]:
y_pred_ada_test = pipeline2.predict(X_test)
overall_accuracy = (y_pred_ada_test == y_test).mean().mean()

print('Average overall accuracy {0:.2f}% \n'.format(overall_accuracy*100))

In [ ]:
y_pred_grid_pd = pd.DataFrame(y_pred_ada_test, columns = y_test.columns)
for column in y_test.columns:
    print('------------------------------------------------------\n')
    print('FEATURE: {}\n'.format(column))
    print(classification_report(y_test[column],y_pred_grid_pd[column]))

### 9. 导出模型为 pickle file

In [ ]:
filename = 'classifier.pkl'
pickle.dump(cv, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
使用资源 (Resources)文件里附带的模板文件编写脚本，运行上述步骤，创建一个数据库，并基于用户指定的新数据集输出一个模型。